In [1]:
# 필요한 라이브러리 설치:
!pip install google-play-scraper
!pip install konlpy
!pip uninstall JPype1-py3
!pip install JPype1


In [2]:
import re
import json
from google_play_scraper import reviews, Sort
from konlpy.tag import Okt
from collections import Counter

def fetch_reviews(app_id, total_reviews=1000):
    """Google Play Store에서 total_reviews 수 만큼 리뷰를 가져옵니다."""
    all_reviews = []
    count = 100  # 한 번에 요청할 리뷰 개수
    token = None

    while len(all_reviews) < total_reviews:
        result, token = reviews(
            app_id,
            lang='ko',
            country='kr',
            sort=Sort.NEWEST,
            count=count,
            continuation_token=token
        )
        all_reviews.extend(result)
        if token is None:
            break  # 더 이상 리뷰가 없으면 종료
    return all_reviews[:total_reviews]

def analyze_reviews(review_list):
    """리뷰 목록에서 텍스트를 결합하여 명사를 추출한 후 빈도 상위 10개 단어를 반환합니다."""
    okt = Okt()
    # 모든 리뷰의 텍스트를 하나의 문자열로 결합
    all_text = " ".join([review['content'] for review in review_list])
    # 한글과 공백을 제외한 문자 제거
    all_text = re.sub(r'[^가-힣\s]', '', all_text)
    
    # 명사 추출
    nouns = okt.nouns(all_text)
    
    # 불용어 처리 (필요에 따라 추가 가능)
    stopwords = set(['이', '그', '저', '것', '들', '의', '있', '하', '되', '수', '때문', '절대', '원래', '관련', '아예', '읍니', '만하','에드','리기'])
    # 한 글자 단어 및 불용어 제거
    nouns = [noun for noun in nouns if noun not in stopwords and len(noun) > 1]
    
    counter = Counter(nouns)
    top_10 = counter.most_common(10)
    return top_10

def group_reviews_by_rating(review_list, target_count=10):
    """
    리뷰 목록을 평점(1~5점)별로 그룹화하여 각 평점에서 최대 target_count개의 리뷰만 반환합니다.
    """
    grouped = {1: [], 2: [], 3: [], 4: [], 5: []}
    for review in review_list:
        score = review.get('score')
        if score in grouped and len(grouped[score]) < target_count:
            grouped[score].append(review)
    return grouped

if __name__ == "__main__":
    # 중고나라 앱의 패키지명 (실제 앱의 패키지명으로 변경하세요)
    app_id = 'com.elz.secondhandstore'
    
    print("리뷰를 수집 중입니다...")
    reviews_data = fetch_reviews(app_id, total_reviews=1000)
    total_reviews = len(reviews_data)
    print(f"{total_reviews}개의 리뷰를 수집했습니다.")
    
    # 전체 리뷰에서 평점별 리뷰 개수를 계산
    overall_counts = Counter(review['score'] for review in reviews_data)
    
    print("\n전체 리뷰 평점별 개수:")
    for score in range(1, 6):
        count_for_score = overall_counts.get(score, 0)
        print(f"{score}점 리뷰: {count_for_score}개")
    
    print(f"\n1점대 리뷰 총 개수: {overall_counts.get(1, 0)}개")
    
    # 평점별로 최대 10개씩 리뷰 그룹화 (분석용)
    grouped_reviews = group_reviews_by_rating(reviews_data, target_count=10)
    
    analysis_result = {}
    
    # 각 평점 그룹별로 리뷰 분석 수행 및 결과 저장
    for score in sorted(grouped_reviews.keys()):
        reviews_for_score = grouped_reviews[score]
        print(f"\n평점 {score}점 리뷰 {len(reviews_for_score)}개 분석:")
        if reviews_for_score:
            top_words = analyze_reviews(reviews_for_score)
            print("가장 많이 나온 단어 TOP 10:")
            for rank, (word, freq) in enumerate(top_words, start=1):
                print(f"{rank}위: {word} - {freq}회")
            # JSON 출력을 위해 tuple을 dict 형태로 변환
            analysis_result[str(score)] = {
                "review_count": len(reviews_for_score),
                "top_words": [{"word": word, "frequency": freq} for word, freq in top_words]
            }
        else:
            print("해당 평점의 리뷰가 없습니다.")
            analysis_result[str(score)] = {
                "review_count": 0,
                "top_words": []
            }
    
    # 최종 결과를 JSON 형태로 구성
    result_data = {
        "total_reviews": total_reviews,
        "overall_counts": {str(score): overall_counts.get(score, 0) for score in range(1, 6)},
        "analysis": analysis_result
    }
    


리뷰를 수집 중입니다...
1000개의 리뷰를 수집했습니다.

전체 리뷰 평점별 개수:
1점 리뷰: 284개
2점 리뷰: 35개
3점 리뷰: 61개
4점 리뷰: 92개
5점 리뷰: 528개

1점대 리뷰 총 개수: 284개

평점 1점 리뷰 10개 분석:
가장 많이 나온 단어 TOP 10:
1위: 문자 - 6회
2위: 사용 - 5회
3위: 거래 - 5회
4위: 업데이트 - 5회
5위: 생각 - 2회
6위: 사진 - 2회
7위: 보기 - 2회
8위: 중고 - 2회
9위: 판매 - 2회
10위: 전화 - 2회

평점 2점 리뷰 10개 분석:
가장 많이 나온 단어 TOP 10:
1위: 업데이트 - 4회
2위: 버그 - 3회
3위: 알림 - 3회
4위: 화면 - 3회
5위: 상품 - 3회
6위: 물건 - 3회
7위: 문제 - 3회
8위: 고객 - 2회
9위: 시간 - 2회
10위: 제때 - 2회

평점 3점 리뷰 10개 분석:
가장 많이 나온 단어 TOP 10:
1위: 사진 - 6회
2위: 채팅 - 5회
3위: 어플 - 4회
4위: 판매 - 3회
5위: 수정 - 3회
6위: 문의 - 3회
7위: 구매 - 3회
8위: 사용 - 3회
9위: 완료 - 2회
10위: 중고나라 - 2회

평점 4점 리뷰 10개 분석:
가장 많이 나온 단어 TOP 10:
1위: 사용 - 2회
2위: 카페 - 2회
3위: 쿠폰 - 2회
4위: 다만 - 1회
5위: 가게 - 1회
6위: 소개 - 1회
7위: 글자 - 1회
8위: 제한 - 1회
9위: 어플 - 1회
10위: 횟수 - 1회

평점 5점 리뷰 10개 분석:
가장 많이 나온 단어 TOP 10:
1위: 업데이트 - 2회
2위: 처음 - 1회
3위: 구매 - 1회
4위: 최근 - 1회
5위: 버벅거림 - 1회
6위: 개발 - 1회
7위: 다시 - 1회
8위: 세상 - 1회
9위: 배달 - 1회
10위: 일주일 - 1회


In [3]:
    print("\nJSON 형태 결과:")
    print(json.dumps(result_data, ensure_ascii=False, indent=2))


JSON 형태 결과:
{
  "total_reviews": 1000,
  "overall_counts": {
    "1": 284,
    "2": 35,
    "3": 61,
    "4": 92,
    "5": 528
  },
  "analysis": {
    "1": {
      "review_count": 10,
      "top_words": [
        {
          "word": "문자",
          "frequency": 6
        },
        {
          "word": "사용",
          "frequency": 5
        },
        {
          "word": "거래",
          "frequency": 5
        },
        {
          "word": "업데이트",
          "frequency": 5
        },
        {
          "word": "생각",
          "frequency": 2
        },
        {
          "word": "사진",
          "frequency": 2
        },
        {
          "word": "보기",
          "frequency": 2
        },
        {
          "word": "중고",
          "frequency": 2
        },
        {
          "word": "판매",
          "frequency": 2
        },
        {
          "word": "전화",
          "frequency": 2
        }
      ]
    },
    "2": {
      "review_count": 10,
      "top_words": [
        {
         